# GlaxoSmithKline - Accumulations


In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("GSK_CH_2017_EFPIA_HCPO_Disclosure_Report.pdf", pages='all', lattice=True, multiple_tables=True)

In [3]:
df = pd.concat(df_list)

## Format Table

In [4]:
df_export = df.copy()

#Remove Duplicates
df_export = df_export.drop_duplicates()

#Shift columns
column = df_export.columns[0]
df_export[df_export[column].isna()] = df_export[df_export[column].isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[8:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#Remove empty rows
#df_export = df_export.dropna(subset=['total'], how='all')

#Remove "EFPIA REPORTDate"
df_export = df_export[~df_export['type'].str.contains("EFPIA REPORTDate", na=False)]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value for Research', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Replace "N/A"
df_export = df_export.replace("N/A", np.NaN)

#Convert to Numbers
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#Shift RND from Sponsorship to total
df_export.loc[df_export.type == 'rnd', 'total'] = df_export.sponsorship
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'glaxosmithkline')

Total nicht Summe der Werte
saved


In [63]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')